# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

import uuid
import random
from datetime import datetime

from clickhouse_driver import Client
from pymongo import MongoClient
import psycopg2
from bson.binary import UuidRepresentation
import psycopg2.extras
psycopg2.extras.register_uuid()

<psycopg2._psycopg.type 'UUID' at 0x7050af0f2480>

# Генерирование данных

In [2]:
BATCH_SIZE = 1000
COUNT_BATCH = 1000000//BATCH_SIZE
COUNT_INSERT = 1000000

ROWS = ['user_id', 'film_id', 'point', 'timestamp']

films = [uuid.uuid4() for i in range(1000)]

def generate_dict():
    return {
        'user_id': uuid.uuid4(),
        'film_id': random.choice(films),
        'point': random.randint(0, 10), 
        'timestamp': datetime.now()
    }
    
def generate():
    return (
        uuid.uuid4(),
        random.choice(films),
        random.randint(0, 10),
        datetime.now()
        )

def generate_many_random_data():
    batch = []
    for i in range(BATCH_SIZE):
        batch.append(generate())
   
    return batch


def generate_many_random_data_dict():
    batch = []
    for i in range(BATCH_SIZE):
        batch.append(generate_dict())
   
    return batch

In [3]:
generate_many_random_data_dict()[0]

{'user_id': UUID('c36f7e3c-5cb0-4b51-8fc3-1fc450e5b6f8'),
 'film_id': UUID('74d8617c-a75d-46de-8808-2efce02fb040'),
 'point': 0,
 'timestamp': datetime.datetime(2024, 3, 24, 17, 6, 10, 94280)}

# Подключение

In [2]:
cl_mongo = MongoClient('localhost', 27017,uuidRepresentation='standard')

In [5]:
connection_pg = psycopg2.connect(dsn="dbname=postgres_db user=postgres password=postgres host=localhost port=5432")
cursor_pg = connection_pg.cursor()

In [6]:
clientCH = Client(host='localhost') 

# Создание таблиц

In [8]:
mongo_db = cl_mongo["test_database"]
mongo_col = mongo_db["test_points"]

In [8]:
cursor_pg.execute(
"""
    CREATE TABLE IF NOT EXISTS test_points(
        user_id UUID,
        film_id UUID,
        point SMALLINT,
        timestamp TIMESTAMP
    );
"""
)
connection_pg.commit()

In [9]:
clientCH.execute('CREATE DATABASE IF NOT EXISTS test_database;')

init = """CREATE TABLE IF NOT EXISTS test_database.test_points (
    user_id UUID,
    film_id UUID,
    point Int16,
    timestamp TIMESTAMP
)
Engine=MergeTree()
ORDER BY (user_id, film_id);"""
clientCH.execute(init)

[]

# Insert батчами

In [10]:
df = pd.DataFrame()

In [11]:
df['num'] = pd.Series([x for x in range(COUNT_BATCH)])

## 1. Mongo

In [12]:
def insert_b_mongo(i):
    data = generate_many_random_data_dict()
    now = datetime.now()
    x = mongo_col.insert_many(data)
    later = datetime.now()
    return (later-now).total_seconds()

In [13]:
df['Mongo ib'] = df['num'].progress_apply(insert_b_mongo)

100%|██████████████████████████████████████| 1000/1000 [00:08<00:00, 122.55it/s]


## 2. Clickhouse

In [14]:
def insert_b_clichouse(i):
    data = generate_many_random_data()
    now = datetime.now()
    clientCH.execute('INSERT INTO test_database.test_points (user_id, film_id, point, timestamp) VALUES',
                     data)
    later = datetime.now()
    return (later-now).total_seconds()

In [19]:
df['clichouse ib'] = df['num'].progress_apply(insert_b_clichouse)

100%|██████████████████████████████████████| 1000/1000 [00:05<00:00, 184.15it/s]


## 3. Postgres

In [16]:
def insert_b_pg(i):
    data = generate_many_random_data()
    now = datetime.now()
    cursor_pg.executemany("INSERT INTO test_points (user_id, film_id, point, timestamp) VALUES(%s,%s,%s,%s)", data)
    connection_pg.commit()
    later = datetime.now()
    return (later-now).total_seconds()

In [17]:
df['postgres ib'] = df['num'].progress_apply(insert_b_pg)

100%|███████████████████████████████████████| 1000/1000 [00:39<00:00, 25.26it/s]


## Результаты

In [18]:
df[['Mongo ib','clichouse ib','postgres ib']].describe()

,Mongo ib,clichouse ib,postgres ib
count,1000.000000,1000.000000,1000.000000
mean,0.005751,0.003123,0.036680
std,0.000489,0.000440,0.003953
min,0.005293,0.002234,0.027856
25%,0.005551,0.002792,0.034152
50%,0.005659,0.003096,0.036255
75%,0.005800,0.003388,0.038790
max,0.017402,0.005394,0.067338


In [20]:
del df

In [21]:
import gc
gc.collect()

16

# Insert по одному

In [22]:
df = pd.DataFrame()
df['num'] = pd.Series([x for x in range(COUNT_INSERT)])

## 1. Mongo

In [23]:
def insert_mongo(i):
    data = generate_dict()
    now = datetime.now()
    x = mongo_col.insert_one(data)
    later = datetime.now()
    return (later-now).total_seconds()

In [24]:
df['Mongo'] = df['num'].progress_apply(insert_mongo)

100%|███████████████████████████████| 1000000/1000000 [02:28<00:00, 6735.20it/s]


## 2. Clickhouse

def insert_b_clichouse(i):
    data = generate_many_random_data()
    now = datetime.now()
    clientCH.execute('INSERT INTO test_database.test_points (user_id, film_id, point, timestamp) VALUES',
                     data)
    later = datetime.now()
    return (later-now).total_seconds()

df['clichouse'] = df['num'].progress_apply(insert_clichouse)

## 3. Postgres

In [25]:
def insert_pg(i):
    data = generate()
    now = datetime.now()
    cursor_pg.execute("INSERT INTO test_points (user_id, film_id, point, timestamp) VALUES(%s,%s,%s,%s)", data)
    connection_pg.commit()
    later = datetime.now()
    return (later-now).total_seconds()

In [26]:
df['postgres'] = df['num'].progress_apply(insert_pg)

100%|████████████████████████████████| 1000000/1000000 [56:08<00:00, 296.91it/s]


## Результаты

In [27]:
df[['Mongo','postgres']].describe()

,Mongo,postgres
count,1000000.000000,1000000.000000
mean,0.000138,0.003229
std,0.000048,0.001594
min,0.000091,0.000264
25%,0.000113,0.002041
50%,0.000122,0.002848
75%,0.000151,0.004529
max,0.007041,0.011811


# Select

In [11]:
films = mongo_col.distinct("film_id")

In [36]:
df = pd.DataFrame()
df['num'] = pd.Series([x for x in range(1000)])

## 1. Mongo

In [34]:
cursor = mongo_col.aggregate([
    { 
        '$match': { 'film_id': { '$in': [ random.choice(films) ] } } 
    },
    { '$group' : {'_id': None , 'avg_val':{'$avg':"$point"}}}
])

In [35]:
for document in cursor:
    print(document)

{'_id': None, 'avg_val': 5.047762694821518}


In [39]:
def select_mongo(i):
    f = random.choice(films)
    now = datetime.now()
    cursor = mongo_col.aggregate([{ 
            '$match': { 'film_id': { '$in': [ f ] } } 
        },
        { '$group' : {'_id': None , 'avg_val':{'$avg':"$point"}}}
    ])
    later = datetime.now()
    return (later-now).total_seconds()

In [40]:
df['Mongo'] = df['num'].progress_apply(select_mongo)

100%|███████████████████████████████████████| 1000/1000 [06:30<00:00,  2.56it/s]


## 2. Clickhouse

In [41]:
f = random.choice(films)
clientCH.execute(f"SELECT avg(point) FROM test_database.test_points where film_id='{f}'")

[(5.057045343734764,)]

In [42]:
def select_clichouse(i):
    f = random.choice(films)
    now = datetime.now()
    clientCH.execute(f"SELECT avg(point) FROM test_database.test_points where film_id='{f}'")
    later = datetime.now()
    return (later-now).total_seconds()

In [43]:
df['clichouse'] = df['num'].progress_apply(select_clichouse)

100%|██████████████████████████████████████| 1000/1000 [00:07<00:00, 134.01it/s]


## 3. Postgres

In [53]:
cursor_pg.execute("ROLLBACK")
connection_pg.commit()

In [55]:
f = random.choice(films)
cursor_pg.execute("SELECT avg(point) FROM test_points where film_id = %(film_id)s;", {"film_id": f})
cursor_pg.fetchone()

(Decimal('5.0114770459081836'),)

In [56]:
def select_pg(i):
    f = random.choice(films)
    now = datetime.now()
    cursor_pg.execute("SELECT avg(point) FROM test_points where film_id = %(film_id)s;", {"film_id": f})
    cursor_pg.fetchone()
    later = datetime.now()
    return (later-now).total_seconds()

In [57]:
df['postgres'] = df['num'].progress_apply(select_pg)

100%|███████████████████████████████████████| 1000/1000 [00:25<00:00, 38.96it/s]


In [58]:
df[['Mongo','clichouse','postgres']].describe()

,Mongo,clichouse,postgres
count,1000.000000,1000.000000,1000.000000
mean,0.389886,0.007394,0.025489
std,0.027723,0.001254,0.001172
min,0.325447,0.004857,0.023532
25%,0.373211,0.006646,0.024624
50%,0.391570,0.007281,0.025195
75%,0.406921,0.007987,0.026144
max,0.547814,0.024267,0.032464
